In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q git+https://github.com/huggingface/peft.git #peft 에러 때문에 이 버전 써야 함
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

# 데이터셋 정리

데이터는 csv 형태로 저장했다.

In [2]:
import pandas as pd
data = pd.read_csv('train.csv')

데이터셋의 구조는 다음과 같다. \
Q는 질문이고, A는 대답, label은 중립, 정보있음, 말장난의 세 분류를 각 0, 1, 2로 두었다.

In [3]:
data.head()

,Q,A,label
0,기업은행 신입행원 떨어지고,저도 똑같아요,0
1,두번 다시 하기 싫은 헤어짐,헤어짐는 순간은 악몽과 같아요.,1
2,15 여 경제융소 IT SW 직군 취업후기,수업 때 뵌 것 같네요. 축하드립니다.,0
3,정제천이 서강을 구했다!!,5000명 이전근거는 남양주시가 국토부에 제출한 문건입니다이 문건을 남양주 TFT에...,0
4,행시준비생 금융 공기업 준비 조언 부탁드려요,저도 cpa하다가 알아보는데 취준 친구 만나서 묻는게 젤빠르더라구요.,0


In [ ]:
data.iloc[1]

Q          두번 다시 하기 싫은 헤어짐
A        헤어짐는 순간은 악몽과 같아요.
label                    1
Name: 1, dtype: object

각 데이터는 아래와 같은 방법으로 불러올 수 있다.

In [ ]:
data.iloc[1]['Q']

'두번 다시 하기 싫은 헤어짐'

Koalpaca를 이용하여 해당 데이터를 파인튜닝 하기 위해 데이터셋 스타일을 맞춰주자.

In [4]:
import datasets
data =  datasets.dataset_dict.Dataset.from_dict(data)

In [5]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['Q']}\n\n### 답변: {x['A']}<|endoftext|>" }
)
data = data.rename_column('Q', 'instruction')
data = data.rename_column('A', 'output')
data = data.rename_column('label',  'url')

Map:   0%|          | 0/45096 [00:00<?, ? examples/s]

아래 양식대로 생성한다.

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.
- 대신 시간이 오래 걸림... 속이 터집니다.

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

2023-06-12 23:34:42.516563: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 23:34:42.567155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 23:34:43.634819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kjh/anaconda3/envs/koalpaca/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kjh/anaconda3/envs/koalpaca/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kjh/anaconda3/envs/koalpaca/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/kjh/anaconda3/envs/koalpaca/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/kjh/anaconda3/envs/koalpaca/lib/libcudart.so.11.0'), PosixPath('/home/kjh/anaconda3/envs/koalpaca/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

text 데이터의 tokenize

In [11]:

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/45096 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 45096
})

In [156]:
data = data.rename_column('label',  'url')

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


# 학습

In [15]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,

    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=1000, ## 초소량만 학습: 1k step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.padding = True,
model.truncation=True,
trainer.train()

Step,Training Loss
10,5.056600
20,5.067600
30,5.033200
40,5.005900
50,5.092600
60,4.932800
70,4.975400
80,5.134800
90,5.163300
100,5.176200


/home/kjh/anaconda3/envs/koalpaca/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1000, training_loss=5.0564765625, metrics={'train_runtime': 1404.7082, 'train_samples_per_second': 2.848, 'train_steps_per_second': 0.712, 'total_flos': 6904144942202880.0, 'train_loss': 5.0564765625, 'epoch': 0.09})

In [16]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [36]:
input_tokens = tokenizer("### 질문: 등록금 납부하려면 어디로 가야 해?", return_tensors='pt')
input_ids = input_tokens.input_ids.to(0)
input_ids

tensor([[    6,     6,     6,  2438,    29,  7586,  6472,   284,  2604,  2116,
           286, 15983,   547,    34]], device='cuda:0')

In [47]:
model.generate(input_ids[0])

TypeError: PeftModelForCausalLM.generate() takes 1 positional argument but 2 were given

In [25]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to(0), 
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [26]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: "topk_cpu" not implemented for 'Half'